# SimplicialCliqueLifting Tutorial

In [1]:
import rootutils

rootutils.setup_root("./", indicator=".project-root", pythonpath=True)
root_folder = rootutils.find_root()
import omegaconf

from modules.transforms.liftings.graph2simplicial import SimplicialCliqueLifting
from modules.io.load.loaders import GraphLoader

### Import Dataset Config

Either we keep yaml config files and provide a brief overview of them, or we build the required config files by hand in these tutorials. (I prefer the former option.)

In [2]:
dataset_name = "cocitation_cora"
dataset_config = omegaconf.OmegaConf.load(
    f"{root_folder}/configs/dataset/{dataset_name}.yaml"
).parameters
dataset_config

{'data_domain': 'graph', 'data_type': 'cocitation', 'data_name': 'Cora', 'data_dir': '${oc.env:PROJECT_ROOT}/datasets/${parameters.data_domain}/${parameters.data_type}', 'data_split_dir': '${oc.env:PROJECT_ROOT}/datasets/data_splits/${parameters.data_name}', 'num_features': 1433, 'num_classes': 7, 'task': 'classification', 'loss_type': 'cross_entropy', 'monitor_metric': 'accuracy', 'task_level': 'node', 'data_seed': 0, 'split_type': 'random', 'k': 10, 'train_prop': 0.5}

### Import Transform Config

Same dilemma as before, yaml files or dicts within tutorials.

In [4]:
lifting_type = "graph2simplicial"
id_lifting = "simplicial_clique"
transform_config = {
    "lifting": omegaconf.OmegaConf.load(
        f"{root_folder}/configs/transforms/topological_liftings/{lifting_type}/{id_lifting}.yaml"
    )
    # other transforms (e.g. data manipulations, feature liftings) can be added here
}
print(transform_config)

{'lifting': {'_target_': 'modules.transforms.data_transform.DataTransform', 'transform_type': 'lifting', 'transform_name': 'SimplicialCliqueLifting', 'complex_dim': '${oc.select:parameters.max_dim_if_lifted,2}', 'preserve_edge_attr': '${oc.select:parameters.preserve_edge_attr_if_lifted,False}', 'signed': True, 'feature_lifting': 'projection'}}


### Load and Transform the Dataset

In [5]:
dataset = GraphLoader(dataset_config, transform_config).load()

Transform parameters are the same, using existing data_dir: /Users/gbg141/Documents/TopoProjectX/challenge-icml-2024/datasets/graph/cocitation/Cora/lifting/4278182681


/opt/miniconda3/envs/topox/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:301: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [6]:
dataset

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[1354], val_mask=[677], test_mask=[677], incidence_0=[1, 2708], down_laplacian_0=[2708, 2708], up_laplacian_0=[2708, 2708], adjacency_0=[2708, 2708], hodge_laplacian_0=[2708, 2708], incidence_1=[2708, 5278], down_laplacian_1=[5278, 5278], up_laplacian_1=[5278, 5278], adjacency_1=[5278, 5278], hodge_laplacian_1=[5278, 5278], incidence_2=[5278, 1630], down_laplacian_2=[1630, 1630], up_laplacian_2=[1630, 1630], adjacency_2=[1630, 1630], hodge_laplacian_2=[1630, 1630], shape=[3], x_0=[2708, 1433], x_1=[5278, 1433], x_2=[1630, 1433])

### Create a Neural Network Model

In [7]:
from topomodelx.nn.simplicial.san import SAN
import torch


class Network(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, n_layers=1):
        super().__init__()
        self.base_model = SAN(
            in_channels=in_channels,
            hidden_channels=hidden_channels,
            n_layers=n_layers,
        )
        self.linear = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, laplacian_up, laplacian_down):
        x = self.base_model(x, laplacian_up, laplacian_down)
        x = self.linear(x)
        return torch.sigmoid(x)

### Run the Model

In [8]:
n_layers = 2
in_channels = dataset_config["num_features"]
hidden_channels = 32
out_channels = dataset_config["num_classes"]

model = Network(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    out_channels=out_channels,
    n_layers=n_layers,
)

In [9]:
y_hat = model(dataset.x_1, dataset.up_laplacian_1, dataset.down_laplacian_1)